## 에스엘 재무제표 합치기
- 재무제표 
- 손익 계산서


## 제무재표 합치기

In [1]:
import warnings
warnings.filterwarnings("ignore", category=UserWarning, module="openpyxl")
## 그냥 데이터프레임 가져올때 뜨는 경고창 무시하는 코드

In [2]:
## 정정 이전 재무제표 

In [3]:
import pandas as pd
import re
from functools import reduce

file_paths = [
    r"C:\Users\qjadl\Desktop\KDT10\EX_DATA_ANALYSYS\MINI_PROJECT\data\[HL만도]분기보고서_재무제표(2015.11.16)_ko.xls",
    r"C:\Users\qjadl\Desktop\KDT10\EX_DATA_ANALYSYS\MINI_PROJECT\data\[HL만도]분기보고서_재무제표(2016.05.16)_ko.xls",
    r"C:\Users\qjadl\Desktop\KDT10\EX_DATA_ANALYSYS\MINI_PROJECT\data\[HL만도]분기보고서_재무제표(2016.11.14)_ko.xls",
    r"C:\Users\qjadl\Desktop\KDT10\EX_DATA_ANALYSYS\MINI_PROJECT\data\[HL만도]분기보고서_재무제표(2017.05.15)_ko.xls",
    r"C:\Users\qjadl\Desktop\KDT10\EX_DATA_ANALYSYS\MINI_PROJECT\data\[HL만도]분기보고서_재무제표(2017.11.14)_ko.xls",
    r"C:\Users\qjadl\Desktop\KDT10\EX_DATA_ANALYSYS\MINI_PROJECT\data\[HL만도]분기보고서_재무제표(2018.05.15)_ko.xls",
    r"C:\Users\qjadl\Desktop\KDT10\EX_DATA_ANALYSYS\MINI_PROJECT\data\[HL만도]분기보고서_재무제표(2018.11.14)_ko.xls",
    r"C:\Users\qjadl\Desktop\KDT10\EX_DATA_ANALYSYS\MINI_PROJECT\data\[HL만도]분기보고서_재무제표(2019.05.15)_ko.xls",
    r"C:\Users\qjadl\Desktop\KDT10\EX_DATA_ANALYSYS\MINI_PROJECT\data\[HL만도]분기보고서_재무제표(2019.11.14)_ko.xls",
    r"C:\Users\qjadl\Desktop\KDT10\EX_DATA_ANALYSYS\MINI_PROJECT\data\[HL만도]분기보고서_재무제표(2020.05.15)_ko.xls",
    r"C:\Users\qjadl\Desktop\KDT10\EX_DATA_ANALYSYS\MINI_PROJECT\data\[HL만도]분기보고서_재무제표(2020.11.16)_ko.xls",
    r"C:\Users\qjadl\Desktop\KDT10\EX_DATA_ANALYSYS\MINI_PROJECT\data\[HL만도]분기보고서_재무제표(2021.05.17)_ko.xls",
    r"C:\Users\qjadl\Desktop\KDT10\EX_DATA_ANALYSYS\MINI_PROJECT\data\[HL만도]분기보고서_재무제표(2021.11.15)_ko.xls",
    r"C:\Users\qjadl\Desktop\KDT10\EX_DATA_ANALYSYS\MINI_PROJECT\data\[HL만도]분기보고서_재무제표(2022.05.16)_ko.xls",
    r"C:\Users\qjadl\Desktop\KDT10\EX_DATA_ANALYSYS\MINI_PROJECT\data\[HL만도]분기보고서_재무제표(2022.11.14)_ko.xls",
    r"C:\Users\qjadl\Desktop\KDT10\EX_DATA_ANALYSYS\MINI_PROJECT\data\[HL만도]분기보고서_재무제표(2023.05.15)_ko.xls",
]

dfs = []
for path in file_paths:
    df = pd.read_excel(path, sheet_name="재무상태표", skiprows=5)

    # 1️⃣ 첫 번째 컬럼명을 '항목'으로 변경
    df.rename(columns={df.columns[0]: "항목"}, inplace=True)

    # 2️⃣ 항목 컬럼 정리: 공백 + 괄호 속 텍스트 + 끝 숫자 제거
    df["항목"] = (
        df["항목"]
        .astype(str)
        .str.strip()
        .str.replace(r"\s+", "", regex=True)      # 모든 공백 제거
        .str.replace(r"\([^)]*\)", "", regex=True)  # 괄호와 그 안 내용 제거
        .str.replace(r"\d+$", "", regex=True)       # 끝 숫자 제거
    )

    # 3️⃣ 파일명에서 날짜 추출 (예: 2023.05.15)
    date_tag = path.split("재무제표(")[-1].split(")_ko.")[0]

    # 4️⃣ 항목 + 첫 번째 값 컬럼(1개)만 사용
    first_value_col = df.columns[1]
    df = df[["항목", first_value_col]].copy()

    # 5️⃣ 값 컬럼명을 날짜로 변경
    df.rename(columns={first_value_col: date_tag}, inplace=True)

    # 6️⃣ 같은 항목 이름끼리 먼저 합치기 (파일 내부 중복 제거)
    df = df.groupby("항목", as_index=False).sum()

    dfs.append(df)

# ✅ 모든 df를 '항목' 기준으로 outer merge
df_merged = reduce(lambda left, right: pd.merge(left, right, on="항목", how="outer"), dfs)

# ✅ 병합 후에도 혹시 중복 있으면 한 번 더 정리
df_merged = df_merged.groupby("항목", as_index=False).sum()

# ✅ 보기 좋게 정렬
df_merged = df_merged.sort_values("항목").reset_index(drop=True)

# ✅ 엑셀로 저장
save_path = r"C:\Users\qjadl\Desktop\KDT10\EX_DATA_ANALYSYS\MINI_PROJECT\OP_DATA\test1.xlsx"
df_merged.to_excel(save_path, index=False)

print(f"\n✅ 병합 완료! 엑셀로 저장됨 → {save_path}")




✅ 병합 완료! 엑셀로 저장됨 → C:\Users\qjadl\Desktop\KDT10\EX_DATA_ANALYSYS\MINI_PROJECT\OP_DATA\test1.xlsx


In [4]:
import pandas as pd
import re

file_paths = [
    r"C:\Users\qjadl\Desktop\KDT10\EX_DATA_ANALYSYS\MINI_PROJECT\data\[(주)만도]3분기보고서_재무제표(2023.11.14)_ko.xlsx",
    r"C:\Users\qjadl\Desktop\KDT10\EX_DATA_ANALYSYS\MINI_PROJECT\data\[에이치엘만도㈜]1분기보고서_재무제표(2024.05.16)_ko.xlsx",
    r"C:\Users\qjadl\Desktop\KDT10\EX_DATA_ANALYSYS\MINI_PROJECT\data\[에이치엘만도㈜]3분기보고서_재무제표(2024.11.14)_ko.xlsx",
    r"C:\Users\qjadl\Desktop\KDT10\EX_DATA_ANALYSYS\MINI_PROJECT\data\[에이치엘만도㈜]1분기보고서_재무제표(2025.05.15)_ko.xlsx",
]

dfs = []
for i, path in enumerate(file_paths):
    df = pd.read_excel(path, sheet_name="D210005", skiprows=4)
    
    # 1) 첫 번째 컬럼명 변경
    df.rename(columns={df.columns[0]: "항목"}, inplace=True)
    
    # 2) 항목 컬럼 공백 제거
    df["항목"] = (
        df["항목"]
        .astype(str)
        .str.strip()
        .str.replace(r"\s+", "", regex=True)
    )

    # 3) 파일명에서 기준일(날짜) 추출
    date_tag = path.split("재무제표(")[-1].split(")_ko.xlsx")[0]

    # 4) 항목 + 첫 번째 값 컬럼 1개만 추출
    first_value_col = df.columns[1]   # 두 번째 컬럼
    df = df[["항목", first_value_col]].copy()
    df.rename(columns={first_value_col: date_tag}, inplace=True)

    # 🔥 5) 같은 항목 이름끼리 먼저 합치기 (이게 핵심!)
    df = df.groupby("항목", as_index=False).sum()

    dfs.append(df)

# ✅ 항목 기준으로 병합
df_merged = dfs[0]
for df_next in dfs[1:]:
    df_merged = pd.merge(df_merged, df_next, on="항목", how="outer")

# 혹시 병합 후에도 중복 항목이 남아 있으면 한 번 더 합치기 (안전용)
df_merged = df_merged.groupby("항목", as_index=False).sum()

# ✅ 정렬
df_merged = df_merged.sort_values("항목").reset_index(drop=True)

# ✅ (옵션) 컬럼명 정제 – 지금은 이미 날짜 그대로라면 이 부분은 생략 가능
def clean_col(col):
    if isinstance(col, tuple):
        col = "_".join([str(c) for c in col if str(c) != 'nan'])
    col = str(col)
    if "_" in col and re.search(r"\d{4}\.\d{2}\.\d{2}", col):
        return re.findall(r"(\d{4}\.\d{2}\.\d{2})", col)[-1]
    else:
        return col.strip()

df_merged.columns = [clean_col(c) for c in df_merged.columns]

# ✅ 엑셀 저장
save_path = r"C:\Users\qjadl\Desktop\KDT10\EX_DATA_ANALYSYS\MINI_PROJECT\OP_DATA\test2.xlsx"
df_merged.to_excel(save_path, index=False)

print(f"\n✅ 병합 완료! 엑셀로 저장됨 → {save_path}")




✅ 병합 완료! 엑셀로 저장됨 → C:\Users\qjadl\Desktop\KDT10\EX_DATA_ANALYSYS\MINI_PROJECT\OP_DATA\test2.xlsx


In [5]:
import pandas as pd

file1 = r"C:\Users\qjadl\Desktop\KDT10\EX_DATA_ANALYSYS\MINI_PROJECT\OP_DATA\test1.xlsx"
file2 = r"C:\Users\qjadl\Desktop\KDT10\EX_DATA_ANALYSYS\MINI_PROJECT\OP_DATA\test2.xlsx"

df1 = pd.read_excel(file1)
df2 = pd.read_excel(file2)


# 4) '항목' 기준으로 병합
# how='outer'  : 두 파일의 항목 전체 포함 (겹치는 건 한 줄로, 나머진 NaN)
df_merged = pd.merge(df1, df2, on='항목', how='outer')

df_merged = df_merged.sort_values('항목').reset_index(drop=True)

# 6) 엑셀로 저장
save_path = r"C:\Users\qjadl\Desktop\KDT10\EX_DATA_ANALYSYS\MINI_PROJECT\OP_DATA\test.xlsx"
df_merged.to_excel(save_path, index=False)

print(f"\n✅ 병합 완료! '항목' 이름이 같은 것만 묶어서 병합됨 → {save_path}")



✅ 병합 완료! '항목' 이름이 같은 것만 묶어서 병합됨 → C:\Users\qjadl\Desktop\KDT10\EX_DATA_ANALYSYS\MINI_PROJECT\OP_DATA\test.xlsx
